## Cortical Microcircuit Modeling: Optogenetic Perturbations and Network Stability Analysis 
### (Aug. 2025) - updated

### <span style='color:#336C43; font-weight:bold'> Overview: This project investigates the dynamic properties of a biologically-inspired cortical microcircuit using the NetPyNE simulation framework. The study focuses on understanding gain control mechanisms and testing for Inhibition-Stabilized Network (ISN) properties through targeted optogenetic manipulations.</span>

### <span style='color:#2E86C1; font-weight:bold'> Features: 
* *NEURON-based neural simulation via NetPyNE framework*  
* *Biophysical modeling of Hodgkin-Huxley neuron dynamics with realistic membrane properties*  
* *Probabilistic synaptic connectivity with different weight parameters for excitatory and inhibitory connections*  
* *Optogenetic perturbation simulation using current clamp protocols*  
* *Spike data analysis and firing rate calculation with custom Python algorithms*
* *Population dynamics visualization through raster plots, PSTHs, and bar charts*
* *Network stability assessment through paradoxical response testing*  
* *Data-driven parameterization from experimental electrophysiology data*
    



In [ ]:
from netpyne import specs, sim
import pandas as pd

# load data
df = pd.read_csv('project_81_neurons_data.csv')
df = df.sort_values('population_group').reset_index(drop=True)

netParams = specs.NetParams()
soma_area = 1e-5 

# population
counts = df['population_group'].value_counts()
for group, count in counts.items():
    netParams.popParams[group] = {'cellType': group, 'numCells': int(count)}

# cell biophysics
for gid, row in df.iterrows():
    ri_ohm = row['ef__ri'] * 1e6
    gl_val = 1.0 / (ri_ohm * soma_area)
    cm_val = max((row['ef__tau'] * 1e-3 * gl_val) * 1e6, 0.1) # reminder: Cm is in uF/cm2 & tau in ms 
    # *****((((tau = Rm * Cm => Cm = tau / Rm => tau = Cm/gL))))*****
    
    netParams.cellParams[f"cell_{gid}"] = {
        'conds': {'gid': gid},
        'secs': {'soma': {
            'geom': {'diam': 18.8, 'L': 18.8, 'cm': cm_val},
            'mechs': {'hh': {'gnabar': 0.12, 'gkbar': 0.036, 'gl': gl_val, 'el': row['ef__vrest']}}
        }}
    }

# syn mechanism
netParams.synMechParams['exc'] = {'mod': 'Exp2Syn', 'tau1': 0.1, 'tau2': 1.0, 'e': 0}
netParams.synMechParams['inh'] = {'mod': 'Exp2Syn', 'tau1': 0.5, 'tau2': 5.0, 'e': -80}

netParams.connParams['PV->Exc'] = {
    'preConds': {'pop': 'Inh_PV'}, 'postConds': {'pop': ['Exc_S', 'Exc_M']},
    'probability': 0.45, 'weight': 0.02, 'synMech': 'inh', 'delay': 2.0}
netParams.connParams['SOM->All'] = {
    'preConds': {'pop': 'Inh_SOM'}, 'postConds': {'pop': ['Exc_S', 'Exc_M', 'Inh_PV']},
    'probability': 0.35, 'weight': 0.015, 'synMech': 'inh', 'delay': 4.0}
netParams.connParams['Exc->Exc'] = {
    'preConds': {'pop': ['Exc_S', 'Exc_M']}, 'postConds': {'pop': ['Exc_S', 'Exc_M']},
    'probability': 0.15, 'weight': 0.006, 'synMech': 'exc', 'delay': 5.0}

# stimulation
netParams.stimSourceParams['bkg'] = {'type': 'NetStim', 'rate': 120, 'noise': 0.85}
netParams.stimTargetParams['bkg->all'] = {
    'source': 'bkg', 'conds': {'pop': ['Exc_S', 'Exc_M', 'Inh_PV', 'Inh_SOM']}, 
    'weight': 0.045, 'delay': 2.0, 'synMech': 'exc'}

# simulation
simConfig = specs.SimConfig()
simConfig.duration = 1000
simConfig.dt = 0.05
simConfig.filename = 'microcircuit_81_results'
simConfig.saveJson = True
simConfig.saveCsv = True 
simConfig.saveDataInclude = ['simData', 'netParams', 'net']

# Record voltage for a few cells
simConfig.recordCells = [0, 20, 40, 60] 
simConfig.recordTraces = {'V_soma': {'sec': 'soma', 'loc': 0.5, 'var': 'v'}}

# saving
pop_colors = {'Exc_S': "#E74C3C", 'Exc_M': '#EB984E', 'Inh_PV': "#2E86C1", 'Inh_SOM': '#28B463'}

# Raster & Population Histogram
simConfig.analysis['plotRaster'] = {'saveFig': True, 'popColors': pop_colors, 'orderInverse': True, 'spikeHist': 'bin'}

# Power Spectrum
simConfig.analysis['plotRatePSD'] = {'saveFig': True, 'include': ['allCells', 'Exc_S', 'Inh_PV'], 'timeRange': [200, 1000]}

# Connectivity Matrix
simConfig.analysis['plotConn'] = {'saveFig': True, 'feature': 'strength', 'groupBy': 'pop'}

# Spike Stats (Firing rates & regularity)
simConfig.analysis['plotSpikeStats'] = {'saveFig': True, 'stats': ['rate', 'isicv'], 'include': ['allCells']}

# individual neuron voltage
simConfig.analysis['plotTraces'] = {'include': [15, 40, 65, 80], 'saveFig': True}

# Run Simulation
sim.createSimulateAnalyze(netParams = netParams, simConfig = simConfig)


import csv
if sim.rank == 0:
    # Extracts spike times and IDs from the gathered data
    spkts = sim.allSimData['spkt']
    spkids = sim.allSimData['spkid']
    
    with open(simConfig.filename + '_spikes.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['spkt', 'spkid'])
        writer.writerows(zip(spkts, spkids))
    print(f"DONE: Spike data saved to {simConfig.filename}_spikes.csv")


Start time:  2026-01-25 17:45:52.728156




Creating network of 4 cell populations on 1 hosts...: 100%|##########|


  Number of cells on node 0: 81 
  Done; cell creation time = 0.10 s.
Making connections...


  PV->Exc: 100%|##########| Creating synaptic connections for 60/60 postsynaptic cells on node 0 (probabilistic connectivity)
  SOM->All: 100%|##########| Creating synaptic connections for 75/75 postsynaptic cells on node 0 (probabilistic connectivity)
  Exc->Exc: 100%|##########| Creating synaptic connections for 60/60 postsynaptic cells on node 0 (probabilistic connectivity)


  Number of connections on node 0: 1084 
  Done; cell connection time = 0.19 s.
Adding stims...
  Number of stims on node 0: 81 
  Done; cell stims creation time = 0.01 s.
Recording 7 traces of 1 types on node 0

Running simulation using NEURON for 1000.0 ms...
  Done; run time = 3.68 s; real-time ratio: 0.27.

Gathering data...
  Done; gather time = 0.05 s.

Analyzing...
  Cells: 81
  Connections: 1165 (14.38 per cell)
  Spikes: 942 (11.63 Hz)
  Simulated time: 1.0 s; 1 workers
  Run time: 3.68 s
Saving output as microcircuit_81_results_data.json ... 
Finished saving!
  Done; saving time = 0.26 s.
Preparing spike data...
Plotting raster...
Plotting firing rate power spectral density (PSD) ...
Plotting connectivity matrix...
Plotting spike stats...
Plotting recorded cell traces ... cell
  Done; plotting time = 1.59 s

Total time = 5.88 s
DONE: Spike data saved to microcircuit_81_results_spikes.csv


In [ ]:
sim.initialize() 

# laser definition
netParams.stimSourceParams['optogenetic_laser'] = {
    'type': 'IClamp',
    'delay': 400,    
    'dur': 200,      
    'amp': -1.115      
}

# targeted target :D
netParams.stimTargetParams['laser->PV_cells'] = {
    'source': 'optogenetic_laser',
    'conds': {'pop': 'Inh_PV'},  
    'sec': 'soma',
    'loc': 0.5
}


simConfig.saveFig = True
simConfig.filename = 'optogenetic_experiment'

# define exactly what plots we want and where to save them
simConfig.analysis['plotRaster'] = {
    'include': ['allCells'], 
    'saveFig': 'optogenetic_raster.png', # filename
    'timeRange': [0, 1000], 
    'lines': [400, 600], #  matches 'dur': 200
    'showFig': False,
    'popColors': {'Exc_S': "#E74C3C", 'Exc_M': '#EB984E', 'Inh_PV': '#2E86C1', 'Inh_SOM': '#28B463'}
}

simConfig.analysis['plotRatePSD'] = {
    'include': ['allCells', 'Inh_PV', 'Exc_S'], 
    'saveFig': 'optogenetic_psd.png', 
    'showFig': False
}

simConfig.analysis['plotTraces'] = {
    'include': [('Inh_PV', 0)], # watch the laser hit one PV cell
    'saveFig': 'optogenetic_traces.png', 
    'showFig': False
}

#  run the simulation
sim.createSimulateAnalyze(netParams=netParams, simConfig=simConfig)


Start time:  2026-01-25 18:21:23.809774

Start time:  2026-01-25 18:21:23.810548




Creating network of 4 cell populations on 1 hosts...: 100%|##########|


  Number of cells on node 0: 81 
  Done; cell creation time = 0.08 s.
Making connections...


  PV->Exc: 100%|##########| Creating synaptic connections for 60/60 postsynaptic cells on node 0 (probabilistic connectivity)
  SOM->All: 100%|##########| Creating synaptic connections for 75/75 postsynaptic cells on node 0 (probabilistic connectivity)
  Exc->Exc: 100%|##########| Creating synaptic connections for 60/60 postsynaptic cells on node 0 (probabilistic connectivity)


  Number of connections on node 0: 1084 
  Done; cell connection time = 0.25 s.
Adding stims...
  Number of stims on node 0: 96 
  Done; cell stims creation time = 0.01 s.
Recording 4 traces of 1 types on node 0

Running simulation using NEURON for 1000.0 ms...
  Done; run time = 3.82 s; real-time ratio: 0.26.

Gathering data...
  Done; gather time = 0.07 s.

Analyzing...
  Cells: 81
  Connections: 1165 (14.38 per cell)
  Spikes: 917 (11.32 Hz)
  Simulated time: 1.0 s; 1 workers
  Run time: 3.82 s
Saving output as optogenetic_experiment_data.json ... 
Finished saving!
  Done; saving time = 0.34 s.
Preparing spike data...
Plotting raster...
Plotting firing rate power spectral density (PSD) ...
Plotting connectivity matrix...
Plotting spike stats...
Plotting recorded cell traces ... cell
  Done; plotting time = 1.50 s

Total time = 6.08 s


In [28]:
import numpy as np
import matplotlib.pyplot as plt

# extract raw data
spkts = np.array(sim.allSimData['spkt'])
spkids = np.array(sim.allSimData['spkid'])

#  define windows
pre_laser = [200, 400]   # baseline
during_laser = [400, 600] # perturbation

 # gid 
exc_gids = [c.gid for c in sim.net.cells if c.tags['pop'] in ['Exc_S', 'Exc_M']]
pv_gids = [c.gid for c in sim.net.cells if c.tags['pop'] == 'Inh_PV']

# rate calculation
def get_pop_rate(gids, time_range):
    if not gids: return 0
    # Filter spikes by time window
    mask = (spkts >= time_range[0]) & (spkts < time_range[1])
    spikes_in_window = spkids[mask]
    
    # Count how many of those spikes belong to our GID list
    count = np.isin(spikes_in_window, gids).sum()
    
    duration_sec = (time_range[1] - time_range[0]) / 1000.0
    return count / (len(gids) * duration_sec)


rate_control = get_pop_rate(exc_gids, pre_laser)
rate_silenced = get_pop_rate(exc_gids, during_laser)

print(f"Gain Control Analysis")
print(f"Excitatory Rate (Control): {rate_control:.2f} Hz")
print(f"Excitatory Rate (PV Silenced): {rate_silenced:.2f} Hz")

if rate_control > 0:
    gain_change = ((rate_silenced - rate_control) / rate_control) * 100
    print(f"Gain Increase: {gain_change:.1f}%")
else:
    print("Baseline activity too low to calculate gain.")
    import matplotlib.pyplot as plt
import numpy as np

# bar Chart
conditions = ['Control', 'PV Silenced']
rates = [rate_control, rate_silenced]
colors = ['#E74C3C', "#4D0C06"]


plt.figure(figsize=(12, 5))

# plot 1; bar chart
plt.subplot(1, 2, 1)
bars = plt.bar(conditions, rates, color=colors, edgecolor='black', alpha=0.8)
plt.ylabel('Excitatory Firing Rate (Hz)', fontsize=12)
plt.title('Disinhibition Effect (Gain Change)', fontsize=11, fontweight='bold')
plt.ylim(0, max(rates) * 1.3)

# value labels 
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 0.2, f'{yval:.2f} Hz', ha='center', fontweight='bold')

# plot 2; PSTH
plt.subplot(1, 2, 2)

# bin the spikes into 20ms windows 
bin_size = 15 # ms
bins = np.arange(0, 1000 + bin_size, bin_size)
counts, _ = np.histogram(spkts[np.isin(spkids, exc_gids)], bins=bins)

# convert counts to Hz
num_exc = len(exc_gids)
rate_over_time = (counts / num_exc) / (bin_size / 1000.0)
bin_centers = bins[:-1] + bin_size/2

plt.plot(bin_centers, rate_over_time, color='#E74C3C', lw=2, label='Exc Population Rate')
plt.xlim(200, 1000)

# shade the "laser on" window 
plt.axvspan(400, 600, color='blue', alpha=0.15, label='PV Silencing')
plt.axvline(400, color='blue', linestyle='--', alpha=0.5)
plt.axvline(600, color='blue', linestyle='--', alpha=0.5)

plt.xlabel('Time (ms)', fontsize=10)
plt.ylabel('Firing Rate (Hz)', fontsize=10)
plt.title('Population Dynamics during Perturbation', fontsize=11, fontweight='bold')
plt.legend()

plt.tight_layout()
plt.savefig('gain_control_analysis.png', dpi=300)
plt.show()

print("saved to folder as 'gain_control_analysis.png'")

Gain Control Analysis
Excitatory Rate (Control): 6.00 Hz
Excitatory Rate (PV Silenced): 8.92 Hz
Gain Increase: 48.6%
saved to folder as 'gain_control_analysis.png'


## <span style='color:red; font-weight:bold'> What would happen if we 'push' the inhibitory cells?  </span>

In [ ]:
# define the stimulus (push to inhibition)
netParams.stimSourceParams['isn_push'] = {
    'type': 'NetStim',
    'rate': 150,     # Strong targeted stimulus
    'noise': 0.5,
    'number': 1000,
    'start': 500     # Start the push halfway through
}

# target inhibitory PV cells
netParams.stimTargetParams['isn_push->PV'] = {
    'source': 'isn_push',
    'conds': {'pop': 'Inh_PV'}, 
    'weight': 0.05,
    'delay': 1.0,
    'synMech': 'exc' # We are exciting the inhibitory cells
}

# simulation
simConfig.filename = 'isn_experiment_PV_push'
sim.createSimulateAnalyze(netParams=netParams, simConfig=simConfig)


Start time:  2026-01-25 18:24:46.299286




Creating network of 4 cell populations on 1 hosts...: 100%|##########|


  Number of cells on node 0: 81 
  Done; cell creation time = 0.07 s.
Making connections...


  PV->Exc: 100%|##########| Creating synaptic connections for 60/60 postsynaptic cells on node 0 (probabilistic connectivity)
  SOM->All: 100%|##########| Creating synaptic connections for 75/75 postsynaptic cells on node 0 (probabilistic connectivity)
  Exc->Exc: 100%|##########| Creating synaptic connections for 60/60 postsynaptic cells on node 0 (probabilistic connectivity)


  Number of connections on node 0: 1084 
  Done; cell connection time = 0.25 s.
Adding stims...
  Number of stims on node 0: 111 
  Done; cell stims creation time = 0.01 s.
Recording 4 traces of 1 types on node 0

Running simulation using NEURON for 1000.0 ms...
  Done; run time = 3.58 s; real-time ratio: 0.28.

Gathering data...
  Done; gather time = 0.05 s.

Analyzing...
  Cells: 81
  Connections: 1165 (14.38 per cell)
  Synaptic contacts: 1180 (14.57 per cell)
  Spikes: 898 (11.09 Hz)
  Simulated time: 1.0 s; 1 workers
  Run time: 3.58 s
Saving output as isn_experiment_PV_push_data.json ... 
Finished saving!
  Done; saving time = 0.22 s.
Preparing spike data...
Plotting raster...
Plotting firing rate power spectral density (PSD) ...
Plotting connectivity matrix...
Plotting spike stats...
Plotting recorded cell traces ... cell
  Done; plotting time = 1.24 s

Total time = 5.47 s


In [ ]:
# extract data
spkts = np.array(sim.allSimData['spkt'])
spkids = np.array(sim.allSimData['spkid'])

# windows
pre_isn = [200, 500]
post_isn = [500, 800]

# gid 
pv_gids = [c.gid for c in sim.net.cells if c.tags['pop'] == 'Inh_PV']
exc_gids = [c.gid for c in sim.net.cells if c.tags['pop'] in ['Exc_S', 'Exc_M']]

def get_rate(gids, window):
    mask = (spkts >= window[0]) & (spkts < window[1])
    count = np.isin(spkids[mask], gids).sum()
    return count / (len(gids) * (window[1]-window[0])/1000.0)

pv_rate_pre = get_rate(pv_gids, pre_isn)
pv_rate_post = get_rate(pv_gids, post_isn)
exc_rate_pre = get_rate(exc_gids, pre_isn)
exc_rate_post = get_rate(exc_gids, post_isn)


print(f"PV Rate Change: {pv_rate_pre:.2f}Hz -> {pv_rate_post:.2f}Hz")
print(f"Exc Rate Change: {exc_rate_pre:.2f}Hz -> {exc_rate_post:.2f}Hz")

if pv_rate_post < pv_rate_pre:
    print("This circuit is an ISN!")
else:
    print("normal response")

PV Rate Change: 24.44Hz -> 21.78Hz
Exc Rate Change: 7.17Hz -> 6.56Hz
This circuit is an ISN!


## Inhibition Stabilized Networks (ISNs) are cortical circuits where strong inhibitory feedback prevents high-level, unstable recurrent excitation from causing runaway activity.

### Because the Exc cells are now firing less, they stop sending Excitation back to the PV cells.